# DATASCI W261: Machine Learning at Scale 

# MIDS-W261-2017-Quiz-Week01-Habib

## Mohammad J. Habib | jhabib@berkeley.edu

## MIDS 261 | Week 1 | Quiz 1.12.2

# This notebook provides a poor man Hadoop through command-line and python. Please insert the python code by yourself.

# Map

In [1]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re

count = 0
WORD_RE = re.compile(r"[\w']+")
filename = sys.argv[2]
findword = sys.argv[1].lower()

with open (filename, "r") as myfile:
    for line in myfile:
        for match in WORD_RE.findall(line):
            if match.lower() == findword:
                print findword, 1

Overwriting mapper.py


In [2]:
!chmod a+x mapper.py

# Reduce

In [3]:
%%writefile reducer.py
#!/usr/bin/python
import sys

total = 0
current_word = None

for line in sys.stdin:
    word, count = line.replace('\n', '').strip().split(' ')
    if current_word:
        if word == current_word: 
            total += int(count)
        else:
            print current_word, total
            total = int(count)
            current_word = word

    current_word = word

print current_word, total

Overwriting reducer.py


In [4]:
!chmod a+x reducer.py

# Write script to file

In [5]:
%%writefile pGrepCount.sh
ORIGINAL_FILE=$1
FIND_WORD=$2
BLOCK_SIZE=$3
CHUNK_FILE_PREFIX=$ORIGINAL_FILE.split
SORTED_CHUNK_FILES=$CHUNK_FILE_PREFIX*.sorted
usage()
{
    echo Parallel grep
    echo usage: pGrepCount filename word chuncksize
    echo greps file file1 in $ORIGINAL_FILE and counts the number of lines
    echo Note: file1 will be split in chunks up to $ BLOCK_SIZE chunks each
    echo $FIND_WORD each chunk will be grepCounted in parallel
}
#Splitting $ORIGINAL_FILE INTO CHUNKS
split -b $BLOCK_SIZE $ORIGINAL_FILE $CHUNK_FILE_PREFIX
#DISTRIBUTE
for file in $CHUNK_FILE_PREFIX*
do
    #grep -i $FIND_WORD $file|wc -l >$file.intermediateCount &
    ./mapper.py $FIND_WORD $file >$file.intermediateCount &
done
wait
#MERGEING INTERMEDIATE COUNT CAN TAKE THE FIRST COLUMN AND TOTOL...
#numOfInstances=$(cat *.intermediateCount | cut -f 1 | paste -sd+ - |bc)
numOfInstances=$(cat *.intermediateCount | ./reducer.py)
echo "found [$numOfInstances] [$FIND_WORD] in the file [$ORIGINAL_FILE]"

Overwriting pGrepCount.sh


# Run the file

In [6]:
!chmod a+x pGrepCount.sh

Usage: usage: pGrepCount filename word chuncksize

In [7]:
!./pGrepCount.sh LICENSE.txt COPYRIGHT 4k
! rm -rf ./LICENSE.txt.split**

found [copyright 58] [COPYRIGHT] in the file [LICENSE.txt]
